In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dvpl1gtu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dvpl1gtu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9363c232d33f30eea21d98a4a0340448226fdd47adbc760e23b25ec3de32a4f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-xwtydf9r/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
# Implement Parallel Traveling Salesman Problem using OpenMP

In [ ]:
code = """
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>
#include <omp.h>

#define MAX_N 16

int n;
int dist[MAX_N][MAX_N];

int visited[MAX_N];
int curr_path[MAX_N];
int min_path[MAX_N];
int min_cost = INT_MAX;

void tsp(int curr_cost, int curr_pos, int level)
{
    if (level == n) {
        // Visited all cities
        curr_cost += dist[curr_pos][0];
        if (curr_cost < min_cost) {
            min_cost = curr_cost;
            #pragma omp critical
            {
                for (int i = 0; i < n; i++) {
                    min_path[i] = curr_path[i];
                }
            }
        }
        return;
    }
    for (int i = 1; i < n; i++) {
        if (!visited[i]) {
            visited[i] = 1;
            curr_path[level] = i;
            tsp(curr_cost + dist[curr_pos][i], i, level + 1);
            visited[i] = 0;
        }
    }
}

int main()
{
    printf("Enter the number of cities: ");
    scanf("%d", &n);

    printf("Enter the distances between the cities:\\n");
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            scanf("%d", &dist[i][j]);
        }
    }

    visited[0] = 1;
    curr_path[0] = 0;
    tsp(0, 0, 1);

    printf("Path: %d", min_path[0] + 1);
    for (int i = 1; i < n; i++) {
        printf("->%d", min_path[i] + 1);
    }
    printf("->%d\\n", min_path[0] + 1);
    printf("Minimum Cost/Minimum weight Hamiltonian Cycle: %d\\n", min_cost);

    return 0;
}



"""

In [ ]:
text_file = open("travel_salesman.c","w")
text_file.write(code)
text_file.close()

In [ ]:
!gcc -o Travelling_salesman -fopenmp travel_salesman.c

In [ ]:
!./Travelling_salesman

Enter the number of cities: 3
Enter the distances between the cities:
3
5
2
55
6
2
66
2
4
Path: 1->3->2->1
Minimum Cost/Minimum weight Hamiltonian Cycle: 59
